In [1]:
#import requiring module
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings

import scipy.stats as st
import seaborn as sns
import itertools
warnings.filterwarnings("ignore")  # specify to ignore warning messages

In [2]:
#loading the data
sns.set(color_codes=True)
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/submission.csv')
df_copy = test.copy()
df_copy.date = pd.to_datetime(df_copy.date)
df_copy.date = pd.to_datetime(df_copy.date.astype(str) + " " + df_copy.time, format='%Y-%m-%d %H:%M:%S')

In [3]:
#열 순서에 주목. 음수값 삭제 함수에서 열 순서로 구분함.
print(df_copy)

        store_id                date      time     card_id  amount  \
0              0 2016-08-01 00:28:15  00:28:15  bf33518373     125   
1              0 2016-08-01 01:09:58  01:09:58  7a19a3a92f      90   
2              0 2016-08-01 01:47:24  01:47:24  6f9fd7e241     150   
3              0 2016-08-01 17:54:43  17:54:43  8bcf1d61b2     362   
4              0 2016-08-01 18:48:53  18:48:53  6a722ce674     125   
...          ...                 ...       ...         ...     ...   
473387       199 2018-03-30 14:17:59  14:17:59  300d7bc922      65   
473388       199 2018-03-30 19:01:54  19:01:54  3ab757718b      65   
473389       199 2018-03-30 20:08:03  20:08:03  2d8e9e421c      65   
473390       199 2018-03-30 20:11:58  20:11:58  22daeb334e     200   
473391       199 2018-03-31 11:41:18  11:41:18  2e698f3302     500   

        installments  days_of_week  holyday  
0                NaN             0        0  
1                NaN             0        0  
2                NaN 

In [4]:
#log를 적용하기 위해서 음수값들을 과거시점의 양수값과 상계처리. 
#예측값은 총 매출이므로 실제 취소전 거래와 100% 매칭 시켜서 상계할 필요는 없다. 다만 이전 거래가 데이터 이전 시점인 경우 삭제한다.
# Remove negative values from the data set.

def reduce_noise_by_removing_neg_vals(df_copy):
    df_pos = df_copy[df_copy.amount > 0]
    df_neg = df_copy[df_copy.amount < 0]

    start = datetime.now()
    
    #to_record -> df_neg 데이터프레임 내의 행을 반복하기 위한 iterator를 반환
    for nega_i in df_neg.to_records()[:]:
        store_i = nega_i[1]
        date_i = nega_i[2]
        card_i = nega_i[4]
        amt_i = nega_i[5]
        
        #pos 데이터프레임에서 상계대상을 찾기위하여 필터링 
        row_i = df_pos[df_pos.store_id == store_i]
        row_i = row_i[row_i.card_id == card_i]
        row_i = row_i[row_i.amount >= abs(amt_i)]
        row_i = row_i[row_i.date <= date_i]
        
        #nega_i와 비교하여 amount가 정확히 같은 행이 있다면
        if len(row_i[row_i.amount == abs(amt_i)]) > 0:
            row_i = row_i[row_i.amount == abs(amt_i)] #amount가 정확히 같은 행만 골라내고
            matched_row = row_i[row_i.date == max(row_i.date)] #amount가 정확히 같은 행 중 가장 최신의 행을 찾는다.
            # df_pos.loc[matched_row.index, 'amount'] = 0
            df_pos = df_pos.loc[~df_pos.index.isin(matched_row.index), :]#해당 행을 삭제한다.(상계처리)
        #nega_i와 비교하여 amount가 정확히 같은 행이 없고, 대신 amount의 절대값이 큰 행이 있다면   
        elif len(row_i[row_i.amount > abs(amt_i)]) > 0:
            matched_row = row_i[row_i.date == max(row_i.date)]#그중에 가장 최신의 행을 찾는다.
            df_pos.loc[matched_row.index, 'amount'] = matched_row.amount + amt_i #상계처리
   
    end = datetime.now()
    time_took = (end - start).seconds / 60
    
    #굉장히 오래걸린다.
    print(round(time_took, 2))#소요시간 로깅.(둘째자리에서 반올림)
    return df_pos


In [5]:
#오래걸림 실행하지 말것.
#df_pos = reduce_noise_by_removing_neg_vals(df_copy)
#편의를 위해 결과를 저장해 둠.
#df_pos.to_csv('df_pos.csv',sep=',',index=False)

In [6]:
df_pos = pd.read_csv('df_pos.csv')
print(df_pos)

        store_id                 date      time     card_id  amount  \
0              0  2016-08-01 00:28:15  00:28:15  bf33518373     125   
1              0  2016-08-01 01:09:58  01:09:58  7a19a3a92f      90   
2              0  2016-08-01 01:47:24  01:47:24  6f9fd7e241     150   
3              0  2016-08-01 17:54:43  17:54:43  8bcf1d61b2     362   
4              0  2016-08-01 18:48:53  18:48:53  6a722ce674     125   
...          ...                  ...       ...         ...     ...   
457414       199  2018-03-30 14:17:59  14:17:59  300d7bc922      65   
457415       199  2018-03-30 19:01:54  19:01:54  3ab757718b      65   
457416       199  2018-03-30 20:08:03  20:08:03  2d8e9e421c      65   
457417       199  2018-03-30 20:11:58  20:11:58  22daeb334e     200   
457418       199  2018-03-31 11:41:18  11:41:18  2e698f3302     500   

        installments  days_of_week  holyday  
0                NaN             0        0  
1                NaN             0        0  
2        

In [7]:
df_pos.describe()

,store_id,amount,installments,days_of_week,holyday
count,457419.000000,457419.000000,1633.000000,457419.000000,457419.000000
mean,91.080720,158.253230,3.372321,3.016151,0.041546
std,54.978209,372.389229,2.616598,1.961984,0.199550
min,0.000000,1.000000,2.000000,0.000000,0.000000
25%,40.000000,40.000000,2.000000,1.000000,0.000000
50%,80.000000,87.000000,3.000000,3.000000,0.000000
75%,142.000000,170.000000,3.000000,5.000000,0.000000
max,199.000000,100000.000000,24.000000,6.000000,1.000000


In [8]:
# 같은 시간에 일어난 카드매출 집계처리
df = df_pos.copy()
test_groupby_date_store = df.groupby(['date', 'store_id'])['amount', 'holyday'].sum()
test_groupby_date_store = test_groupby_date_store.reset_index()
test_groupby_date_store = test_groupby_date_store.set_index('date')
#test_groupby_date_store.index = pd.to_datetime(test_groupby_date_store.index)
print(test_groupby_date_store.index)
print(test_groupby_date_store)

DatetimeIndex(['2016-08-01 00:02:42', '2016-08-01 00:12:43',
               '2016-08-01 00:20:28', '2016-08-01 00:22:01',
               '2016-08-01 00:24:27', '2016-08-01 00:25:19',
               '2016-08-01 00:28:15', '2016-08-01 00:40:17',
               '2016-08-01 00:57:32', '2016-08-01 01:03:18',
               ...
               '2018-03-31 23:35:46', '2018-03-31 23:37:16',
               '2018-03-31 23:37:58', '2018-03-31 23:40:03',
               '2018-03-31 23:43:16', '2018-03-31 23:44:29',
               '2018-03-31 23:44:55', '2018-03-31 23:57:08',
               '2018-03-31 23:57:46', '2018-03-31 23:59:34'],
              dtype='datetime64[ns]', name='date', length=457393, freq=None)
                     store_id  amount  holyday
date                                          
2016-08-01 00:02:42        96      90        0
2016-08-01 00:12:43       125     260        0
2016-08-01 00:20:28        88      50        0
2016-08-01 00:22:01        88      30        0
2016-08-01 

In [9]:
store_list = test_groupby_date_store.store_id.unique()
store_list.sort()
print(store_list)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199]


In [10]:
# #arima 모델 파라미터범위 결정시 사용한것으로 보임. (최종 모델에서는 사용하지 않음)
# def adf_test(y):
#     # perform Augmented Dickey Fuller test
#     print('Results of Augmented Dickey-Fuller test:')
#     dftest = adfuller(y, autolag='AIC')
#     dfoutput = pd.Series(dftest[0:4], index=['test statistic', 'p-value', '# of lags', '# of observations'])
#     for key, value in dftest[4].items():
#         dfoutput['Critical Value ({})'.format(key)] = value
#     print(dfoutput)

# def ts_diagnostics(y, lags=None, title='', filename=''):
#     '''
#     Calculate acf, pacf, qq plot and Augmented Dickey Fuller test for a given time series
#     '''
#     if not isinstance(y, pd.Series):
#         y = pd.Series(y)

#     # weekly moving averages (5 day window because of workdays)
#     rolling_mean = pd.Series.rolling(y, window=2).mean()
#     rolling_std = pd.Series.rolling(y, window=2).std()

#     fig = plt.figure(figsize=(14, 12))
#     layout = (3, 2)
#     ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
#     acf_ax = plt.subplot2grid(layout, (1, 0))
#     pacf_ax = plt.subplot2grid(layout, (1, 1))
#     qq_ax = plt.subplot2grid(layout, (2, 0))
#     hist_ax = plt.subplot2grid(layout, (2, 1))

#     # time series plot
#     y.plot(ax=ts_ax)
#     rolling_mean.plot(ax=ts_ax, color='crimson')
#     rolling_std.plot(ax=ts_ax, color='darkslateblue')
#     plt.legend(loc='best')
#     ts_ax.set_title(title, fontsize=24)

#     # acf and pacf
#     plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
#     plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5)

#     # qq plot
#     sm.qqplot(y, line='s', ax=qq_ax)
#     qq_ax.set_title('QQ Plot')

#     # hist plot
#     y.plot(ax=hist_ax, kind='hist', bins=25)
#     hist_ax.set_title('Histogram')
#     plt.tight_layout()
#     plt.show()

#     # perform Augmented Dickey Fuller test
#     print('Results of Dickey-Fuller test:')
#     dftest = adfuller(y, autolag='AIC')
#     dfoutput = pd.Series(dftest[0:4], index=['test statistic', 'p-value', '# of lags', '# of observations'])
#     for key, value in dftest[4].items():
#         dfoutput['Critical Value (%s)' % key] = value
#     print(dfoutput)
#     return

In [11]:
# ARIMA 모델의 모수를 찾기 위한 함수 정의
def get_optimal_params(y):
    # Define the p, d and q parameters to take any value between 0 and 1

    param_dict = {}
    for param in pdq:
        try:
            #SARIMAX는 무엇일까?
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            )
            results = mod.fit()
            model = ARIMA(y, order=param)
            results_ARIMA = model.fit(disp=-1)
            results_ARIMA.summary()
            param_dict[results.aic] = param
        except:
            continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params

In [22]:
def arima_main(input_df, sampling_period_days, fcst_period):
    #sampling_period_days로 나누어떨어질 수 있도록 나머지에 해당하는 과거의 행은 버린다.
    input_df = input_df[len(input_df) % sampling_period_days:].resample(str(sampling_period_days) + 'D').sum()
    prob_of_no_sales = len(input_df[(input_df.amount == 0) | (input_df.amount.isna())]) / len(input_df)
    ts_log = np.log(input_df.amount)
    ts_log = ts_log[~ts_log.isin([np.nan, np.inf, -np.inf])]
    
    if len(ts_log) < min_period:
        return None
    if sampling_period_days >= 28:
        expected_return_pct_lending = 0.13 * (100 + 16 + 6.8) / 365
    elif sampling_period_days >= 14:
        expected_return_pct_lending = 0.13 * (100 + 16 + 14) / 365
    else:
        expected_return_pct_lending = 0.13 * (100 + 16 + 6.8) / 365

    expected_loss_pct_lending = 1.00
    optimal_prob = expected_loss_pct_lending / (expected_loss_pct_lending + expected_return_pct_lending)
    optimal_z_score = st.norm.ppf(optimal_prob)

    optimal_params = get_optimal_params(ts_log)
    pdqs[store_i] = optimal_params

    model = ARIMA(ts_log, order=optimal_params)
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(fcst_period)

    fcst_means = fcst[0]
    fcst_stds = fcst[1]
    fcst_i = fcst_means - (fcst_stds * optimal_z_score)
    fcst_i = sum(map(lambda x: np.exp(x) if np.exp(x) > 0 else 0, fcst_i))
    #보정치
    prediction_i = fcst_i * (1 - prob_of_no_sales)
    #예측값 리턴
    return prediction_i

In [18]:
#프로그램 실행 전 실행시 사용되는 전역변수들 정의 (실제로 사용하진 않음.)
sampling_p = 28
predic_len = math.floor(100 / sampling_p)
expected_return_pct_lending = 0.13 * (100 + 16 + 6.8) / 365
expected_loss_pct_lending = 1.00

#optimal Z는 무엇일까?
optimal_prob = expected_loss_pct_lending / (expected_loss_pct_lending + expected_return_pct_lending)
optimal_z_score = st.norm.ppf(optimal_prob)

print(optimal_prob)
print(optimal_z_score)


0.9557475778999738
1.7033379408274


In [19]:
#프로그램 실행 전 실행시 사용되는 전역변수들 정의
mean_period = 2 * 3 #14*2*3 = 84 <- 실행지점 맨 아래쪽 코드 참고.

#다운샘플링 결과로 나온 데이터셋의 행의 수가 min_period 미만이면 arima를 적용하지 않음.
min_period = 6

'''
PACF (편자기상관함수 http://bit.ly/2OeRail)

를 통해 AR(0), AR(1), AR(2)를 확인. 하지만 AIC를 통해 ARIMA모델에 사용될 pqr 값들을 찾을 때 0~2의 값들을 통해 구한 pqr보다 0~1 사이로 찾은 pqr이 더 정확한 예측을 했기 때문에 pqr의 범위를 0~1로 설정 (line 141-160)
'''
max_pdq = 2
p = d = q = range(0, max_pdq) #2는 범위에 포함되지 않음.
pdq = list(itertools.product(p, d, q))
pdqs = dict()


#제출파일 생성 준비
submission_copy = submission.copy()

In [23]:
#실질적인 프로그램 실행 지점
for store_i in store_list[:]:
    #예측 결과 담길 변수 정의
    prediction_i = None
    
    #해당 store_id에 해당하는 행들만 가져옴
    test_df = test_groupby_date_store[test_groupby_date_store.store_id == store_i]
    
    #일자별로 행 집계(리샘플링)
    test_df_daily = test_df.resample('D').sum()
    
    #다운샘플링 크기를 28일로해서 예측해본다. (28*3 = 84)
    prediction_i = arima_main(test_df_daily, sampling_period_days=28, fcst_period=3)
    
    #모델 개발시 21일도 해본것으로 추측할 수 있다.
    # if prediction_i is None:
    #     prediction_i = arima_main(test_df_daily, sampling_period_days=21, fcst_period=4)
    
    #28일로 다운샘플링을 했을때 행이 충분하지 않다면 14일로 다운샘플링 하여 예측해본다. (14 *7 = 98)
    if prediction_i is None:
        prediction_i = arima_main(test_df_daily, sampling_period_days=14, fcst_period=7)
    
    #위와 비슷(7*12 = 84)
    if prediction_i is None:
        prediction_i = arima_main(test_df_daily, sampling_period_days=7, fcst_period=12)
    
    #다운샘플링 모두 실패시 ARIMA를 적용하지 않고 log 평균을 통해 직접 예측치를 계산한다.
    if prediction_i is None:
        
        #14일로 구분짓고, 나누어떨어질 수 있도록 나머지에 해당하는 행은 버린다.
        test_df = test_df_daily[len(test_df_daily) % 14:].resample('14D').sum()
        
        #no_sales 비율을 계산하여 이후 계산할 예측치에 대한 보정값으로 쓴다
        prob_of_no_sales = len(test_df[(test_df.amount == 0) | (test_df.amount.isna())]) / len(test_df)
        
        #로그를 취한다.
        ts_log = ts_log[~ts_log.isin([np.nan, np.inf, -np.inf])]
        ts_log_wkly = np.log(test_df.amount)
        
        #예측값 계산
        estimated_amt = np.exp(ts_log_wkly.mean() - ts_log_wkly.std() * optimal_z_score) * (1 - prob_of_no_sales)
        
        
        prediction_i = estimated_amt * mean_period
    #예측 결과를 제출파일에 저장한다.
    submission_copy.loc[submission_copy['store_id'] == store_i, 'total_sales'] = prediction_i
print(submission_copy)

            store_id  amount  holyday
date                                 
2016-08-01         0    2106        0
2016-08-02         0    1528        0
2016-08-03         0     560        0
2016-08-04         0    1683        0
2016-08-05         0    1686        0
...              ...     ...      ...
2018-03-27         0     255        0
2018-03-28         0     821        0
2018-03-29         0     472        0
2018-03-30         0     694        0
2018-03-31         0    1045        0

[608 rows x 3 columns]
            store_id  amount  holyday
date                                 
2016-08-21         0   28807       16
2016-09-18         0   23770        8
2016-10-16         0   27294        0
2016-11-13         0   28752        0
2016-12-11         0   25977       16
2017-01-08         0   22449       17
2017-02-05         0   24138        7
2017-03-05         0   26465        0
2017-04-02         0   32384        0
2017-04-30         0   29708       29
2017-05-28         0   389

In [21]:
#제출파일 저장
#프로그램 전역변수값에 따라 자동으로 파일명을 변경.
# output_file_name_fmt = '../1st_data/py_4arima_pos_sep_{optimal_p}-{sampling_period}_no_sales_prob&no mean{mean_period}&min_period {min_period}_pdq {max_pdq}.csv'
# output_file_name = output_file_name_fmt.format(optimal_p=round(optimal_prob, 4),
#                                                sampling_period=sampling_p,
#                                                mean_period=mean_period,
#                                                min_period=min_period,
#                                                max_pdq=max_pdq)
print(submission_copy)
output_file_name = 'submissionCopy1st.csv';
submission_copy.to_csv(output_file_name, index=False)
print(output_file_name)

     store_id   total_sales
0           0  68426.936910
1           1  11840.490783
2           2  16208.571184
3           3  26870.446220
4           4  11752.421019
..        ...           ...
195       195  31237.923574
196       196  31340.097798
197       197     57.921562
198       198   1061.787577
199       199  18906.142306

[200 rows x 2 columns]
submissionCopy1st.csv


In [4]:
#downsampling - 여러 행을 집계하여 예측의 단위를 줄임. 
#너무 묶으면 seanality가 사라짐. 예측을 해야되는 기간이 1년이 아니라 100일 이므로 
#월별 Seasonality를 살려주는건 좋음. 요일별 Seasonality는 예측기간(100일)을 고려하면 그 영향도가 작음.
#따라서 다운샘플링 기간을 최소 7일 최대 28일로 진행하는 것이 타당하다고 판단됨. 
#다운샘플링 기준 기간을 7일 14일 28일로 잡은 것은 시계열 데이터의 특성을 최대한 살릴 수 있었다고 생각함.